# FIT5196 Task 1 in Assessment 2
#### Student Name: Boyu Zhang
#### Student ID: 28491300

Date: XX/XX/XXXX

Version: 1.0

Environment: Python 3.6 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas (for dataframe, included in Anaconda Python ) 
* re (for regular expression, included in Anaconda Python 3.6) 
* numpy (for numpy array, included in Anaconda Python 3.6) 



## 1.  Import libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd
import numpy as np
import re

## 2. Load data and exploration


In [2]:
df1 = pd.read_csv('./Input/dataset1_with_error.csv')

In [3]:
#overview of the dataset
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25077 entries, 0 to 25076
Data columns (total 11 columns):
Id                  25077 non-null int64
Title               25077 non-null object
Location            25077 non-null object
ContractType        25077 non-null object
ContractTime        25077 non-null object
Company             21242 non-null object
Category            25077 non-null object
Salary per annum    25077 non-null object
SourceName          25077 non-null object
OpenDate            25077 non-null object
CloseDate           25077 non-null object
dtypes: int64(1), object(10)
memory usage: 2.1+ MB


In [4]:
df1.describe(include=['O'])

,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
count,25077,25077,25077,25077,21242,25077,25077,25077,25077,25077
unique,25077,482,3,3,4879,8,1589,90,2203,2400
top,Manager Risk Regulatory,UK,not available,permanent,UKStaffsearch,IT Jobs,35004,totaljobs.com,20130127T150000,20130526T120000
freq,1,3996,19499,16194,248,7085,1011,5335,27,23


In [5]:
#number of missing values in each column
df1.isnull().sum()

Id                     0
Title                  0
Location               0
ContractType           0
ContractTime           0
Company             3835
Category               0
Salary per annum       0
SourceName             0
OpenDate               0
CloseDate              0
dtype: int64

### Highlight of initial investigation
- There are 9 columns of categorical values, and 2 column of numeric values -- `Id` and `Salary`
- There are a lot of missing values in the `Company` column
- There must be issue with the column `Title` since the most frequent value is like "** *Ton**"which is obviously invalid
- The most frequent value of `Location` is "UK" which suspicious
- There are potential semantic anomalies :
    - unique values can have redundance
    - The violation of two Date
- There can be sytax anomalies:
     - inconistent representation of values in the same column
     - invalid value of categorical values

In [6]:
#first glance of the dataset
df1.head()

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,12612628,Engineering Systems Analyst,Dorking,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20121103T000000,20121203T000000
1,12612830,Stress Engineer Glasgow,Glasgow,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130108T150000,20130408T150000
2,12612844,Modelling and simulation analyst,Hampshire,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130726T150000,20130924T150000
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,27504,cv-library.co.uk,20121214T000000,20130314T000000
4,12613647,"Pioneer, Miser Engineering Systems Analyst",Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20131025T000000,20131224T000000


In [7]:
df1.iloc[3].Title

'Engineering Systems Analyst / Mathematical Modeller'

## 3. Identify issues and fix 

### ID
For the column `ID` we don't have any constraints other than non-redundance. Thus we only check whether all **IDs** is unique.

In [8]:
#test if all Id 
df1.Id.unique().shape[0] == df1.shape[0]

True

### Title
- It's reasonable no matter there are redundant tiltles or not.
- The values should be in valid language

In [9]:
#test whethere there is redundant titles
df1.Title.unique().shape

(25077,)

In [10]:
df1.Title.value_counts()

Manager  Risk Regulatory                                                 1
Senior Project Engineer                                                  1
Payroll Team Manager                                                     1
RMN Nights Darlington ****K                                              1
Senior / Design Manager  Acute Healthcare                                1
Sales Team Leader  New Homes                                             1
CCTV Area Sales Midlands                                                 1
Service Desk  Project CoOrdinator                                        1
Head Chef  new classic pub restaurant near Wilmslow                      1
Exhibition Sales Executive                                               1
Accounts Receivable Coordinator                                          1
Assistant General Manager  Thriving Latin Restaurant  Liverpool ****k    1
Catering Supervisor  **** Per Annum  Wednesday,Thursday & Friday         1
Technical Design Consulta

- It appears there are values that contain invalid characters in the `Title` column 
- It seems some titles also contains the company name or location which could be improved for clearity of data

In [11]:
df1[df1.Title=='**** Ton']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
24644,69193100,**** Ton,Rugby,full_time,permanent,Interaction Recruitment plc,Engineering Jobs,13440,fish4.co.uk,20130815T120000,20131113T120000


In [12]:
df1[df1.Title=='Electronics Engineer, Embedded Software, R D, York, ****']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
2987,65102296,"Electronics Engineer, Embedded Software, R D, ...",North Yorkshire,full_time,permanent,Amrec,Engineering Jobs,37500,thecareerengineer.com,20130926T120000,20131225T120000


In [13]:
df1[df1.Title=='Assistant Manager  South West London Gastro Pub  ****K']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
18230,68706648,Assistant Manager South West London Gastro Pu...,London,not available,not available,Cherryred Recruitment,Hospitality & Catering Jobs,22500,jobs.catererandhotelkeeper.com,20130702T000000,20130831T000000


#### Temporary conclusions:
- After inspecting some records that contains invalid character, there seems no clues how to replace with meaningful values. Thus we think it is better to keep these invalid character, since R****** still tell the length of the word but 'R' will not

### Location 


In [14]:
df1[df1.Location == 'UK']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
9,22933091,Chef de Partie Award Winning Dining Live In ...,UK,not available,not available,Chef Results,Hospitality & Catering Jobs,18000,caterer.com,20120814T000000,20120828T000000
12,23530231,Lead Engineers (Stress),UK,not available,permanent,Gregory Martin International,Engineering Jobs,39504,cv-library.co.uk,20121113T120000,20130211T120000
13,24104334,"Relief Chef de Partie Croydon, Surrey Live in",UK,not available,not available,Towngate Personnel,Hospitality & Catering Jobs,17280,caterer.com,20131029T000000,20131128T000000
26,27739522,Welwyn Chef de Partie does it get any better t...,UK,not available,not available,Clear Selection,Hospitality & Catering Jobs,18000,caterer.com,20130106T120000,20130307T120000
27,27740574,Pastry Chef AL**** ****AA Rosette Restaurant,UK,not available,not available,Clear Selection,Hospitality & Catering Jobs,18996,caterer.com,20120320T120000,20120403T120000
30,29650985,Fine Dining Chefs for rosette restaurants up ...,UK,not available,not available,Clear Selection,Hospitality & Catering Jobs,24000,caterer.com,20131205T150000,20140104T150000
31,30192171,Web Developer,UK,not available,permanent,Indigo 21 Ltd,IT Jobs,35004,cv-library.co.uk,20130305T000000,20130504T000000
39,31989124,Mechanical Engineer Design and Substantiation,UK,not available,permanent,Gregory Martin International,Engineering Jobs,35004,cv-library.co.uk,20131018T120000,20131117T120000
42,32233421,Principle Mechanical Engineer,UK,not available,permanent,Rullion Engineering Ltd,Engineering Jobs,48504,cv-library.co.uk,20130816T120000,20131114T120000
43,32417995,Chef de Partie Award Winning Fine Dining Rest...,UK,not available,not available,Chef Results,Hospitality & Catering Jobs,20496,caterer.com,20131016T150000,20131030T150000


Problem of location seems more complicated:
 - some records has just no detailed `Location` but just "UK"(default filling for none)
 - some has the location data but it is included in the `Title` value and these record also just have the value of `Location` as "UK"

All the other locations seem to be all in the UK, there can be inconsistence the Column of `Location`

### ContractType

In [15]:
#check unique values of the columns
df1.ContractType.unique()

array(['not available', 'full_time', 'part_time'], dtype=object)

According to dta dataset description, the 'not available' should be replaced by 'non_specified'

In [16]:
df1.ContractType.replace('not available','non_specified',inplace=True)

### ContractTime

In [17]:
#check unique values
df1.ContractTime.unique()

array(['permanent', 'not available', 'contract'], dtype=object)

In [18]:
df1.ContractTime.replace('not available','non_specified',inplace=True)

### Company

In [19]:
df1.Company.value_counts()

UKStaffsearch                         248
CVbrowser                             170
Randstad                              169
JOBG8                                 159
Matchtech Group plc.                  150
Chef Results                          137
Penguin Recruitment                   134
Hays                                  115
London4Jobs                           112
COREcruitment International           111
JAM Recruitment Ltd                   109
Computer People                        99
Clear Selection                        92
Monarch Recruitment                    89
Towngate Personnel                     79
Cherryred Recruitment                  69
Senitor Associates                     67
Modis                                  62
Support Services Group                 62
Capita Resourcing                      62
Aspire Data Recruitment                61
Adecco                                 57
Populus Healthcare                     55
Idex Consulting                   

Seems there are some minor irregularities like the inconsitent use of "Limited","Ltd" and "LTD".

In [20]:
# unify all occurences of Ltd

### Company
It is already discovered before that there is large amount of missing values in the `Company` column

In [ ]:
df1.Company

### Category

In [22]:
df1.Category.value_counts()

IT Jobs                             7085
Healthcare & Nursing Jobs           4334
Engineering Jobs                    3458
Accounting & Finance Jobs           3099
Sales Jobs                          2609
Hospitality & Catering Jobs         2124
Teaching Jobs                       1378
PR, Advertising & Marketing Jobs     990
Name: Category, dtype: int64

- No invalid values and no redundance
- Also no inconsistent represnetation of same value

### Salary

In [23]:
df1["Salary per annum"].unique()

array(['24996', '30000', '27504', ..., '17K', '18264', '40368'], dtype=object)

There are inconsiten representations of the salary

In [24]:
p1 = '^\d+$'

In [25]:
non_numeric_salary = df1['Salary per annum'].apply(lambda x: False if re.match(p1,x) else True )

In [26]:
inconsis_salary =df1['Salary per annum'][non_numeric_salary];inconsis_salary.value_counts()

32K                                        7
37996.2 - 41995.8                          5
38K                                        5
35K                                        5
33253.799999999996 - 36754.200000000004    5
30871.199999999997 - 34120.8               5
45K                                        5
28500.0 - 31500.0                          4
26K                                        4
24K                                        4
42750.0 - 47250.0                          4
47K                                        4
19K                                        4
30K                                        4
40K                                        4
20K                                        4
23K                                        4
55K                                        3
50K                                        3
15196.199999999999 - 16795.8               3
27553.8 - 30454.2                          3
26128.8 - 28879.2                          3
23746.1999

There seems to have two type of invalid representations:
1. "90K"
2. "21819.6 - 24116.4"

**For the first one we just transfer the str representation to int.**

**For the second one we pick the mean of the range and round them to the nearest integer.**

In [27]:
#design the regex patterns

In [28]:
numeric_salary = df1['Salary per annum'][~non_numeric_salary];numeric_salary

0        24996
1        30000
2        30000
3        27504
4        24996
5        21000
6        37500
7        15996
8        21996
9        18000
10       39504
11       33504
12       39504
13       17280
14       41496
15       31500
16       32496
17       24000
18       15996
19       26004
20       31500
21       30000
22       32496
23       45000
24       24996
25       21000
26       18000
27       18996
28       30000
29       48000
         ...  
25047    52500
25048    82560
25049    32496
25050    47496
25051    37500
25052    18000
25053    39996
25054    27504
25055    54996
25056    25704
25057    45996
25058    45000
25059    42504
25060    32004
25061    45000
25062    20004
25063    48504
25064    15120
25065    66000
25066    32496
25067    32004
25068    24000
25069    52500
25070    47496
25071    47496
25072    39996
25073    32004
25074    42996
25075    67500
25076    38496
Name: Salary per annum, Length: 24877, dtype: object

In [30]:
int_salary = numeric_salary.astype(int) <= 0
int_salary.sum()

0

Thia proves there is not semantic errors in the row of `Salary per annum`

### SourceName

In [31]:
df1.SourceName.value_counts()

totaljobs.com                     5335
cv-library.co.uk                  4207
careworx.co.uk                    1777
cwjobs.co.uk                      1202
staffnurse.com                    1182
jobsite.co.uk                     1110
caterer.com                       1047
fish4.co.uk                        757
theitjobboard.co.uk                755
jobserve.com                       717
jobs.catererandhotelkeeper.com     640
planetrecruit.com                  460
rengineeringjobs.com               363
salestarget.co.uk                  355
britishjobsonthe.net               332
jobs.guardian.co.uk                301
ukstaffsearch.com                  248
hays.co.uk                         247
strike-jobs.co.uk                  221
technojobs.co.uk                   219
gojobsearch.co.uk                  187
justengineers.net                  179
cvbrowser.com                      170
cityjobs.com                       159
jobg8.com                          145
careerbuilder.com        

Seems no explicit issues.

### OpenData and CloseDate
We assume at the first place that the date form is Y-M-D

In [32]:
df1.OpenDate.head()

0    20121103T000000
1    20130108T150000
2    20130726T150000
3    20121214T000000
4    20131025T000000
Name: OpenDate, dtype: object

The result of above cell proves our assumption, so we can start invetigate whether there are issues in the two columns for date.

In [35]:
p = '\d{4}(\d{2})(\d{2}).(\d{2})(\d{2})(\d{2})'
def is_valid_date(d):
    month_invalid = int(re.match(p,d).group(1)) > 12 
    day_invalid = int(re.match(p,d).group(2)) > 31 #keep it simple fallacious for the moment, for simplicity just assume each month has 31 days
    hr_invalid = int(re.match(p,d).group(3)) > 23
    min_invalid = int(re.match(p,d).group(4)) > 59
    second_invalid = int(re.match(p,d).group(5)) > 59
    if month_invalid or day_invalid or hr_invalid or min_invalid or second_invalid:
        return d
    else:
        return np.nan
    
invalid_open_date = df1.OpenDate.apply(is_valid_date)
invalid_close_date = df1.CloseDate.apply(is_valid_date)

In [36]:
#test
print(invalid_open_date.dropna().shape, invalid_close_date.dropna().shape)

(10,) (0,)


indicates there are 10 invalid date 

inspect the invalid values

In [37]:
invalid_series = invalid_open_date.dropna().values

In [38]:
df1[df1.OpenDate.isin(invalid_series)]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
1102,51593546,Community Nurse South London,The City,non_specified,non_specified,NaN,Healthcare & Nursing Jobs,29004,careworx.co.uk,20131803T120000,20130417T120000
2104,61282006,Deputy Home Care Manager,Barnet,non_specified,non_specified,NaN,Healthcare & Nursing Jobs,20004,careworx.co.uk,20132606T000000,20130710T000000
2839,64796836,"Technical Tester Manual, Automation, Java, SQ...",London,non_specified,permanent,NaN,IT Jobs,19500,theitjobboard.co.uk,20122003T150000,20120519T150000
5707,66935867,"Technical Sales Manager, Pigments Fillers to S...",UK,non_specified,non_specified,Wallace Hind Selection,Sales Jobs,45000,salestarget.co.uk,20121512T150000,20130213T150000
10881,68257270,Desire a Change? London ****K Basic ****K OTE,South East London,non_specified,non_specified,Zest 2 Recruitment,Sales Jobs,13500,salestarget.co.uk,20133004T150000,20130530T150000
11948,68350622,Mechanical Design Engineer (HVAC),Gainsborough,non_specified,permanent,NaN,IT Jobs,39996,technojobs.co.uk,20131908T000000,20131117T000000
15353,68671948,Investment Analyst,Cheshire,non_specified,permanent,Idex Consulting,Accounting & Finance Jobs,24000,totaljobs.com,20121406T120000,20120912T120000
22918,69072305,Scheme Actuary,Hampshire,non_specified,permanent,Sammons Pensions,Accounting & Finance Jobs,54996,professionalpensionsjobs.com,20131509T000000,20131114T000000
23007,69079402,Naval architect/structural engineer,Newcastle Upon Tyne,non_specified,permanent,Workshop Recruitment,Engineering Jobs,33504,cv-library.co.uk,20132901T150000,20130429T150000
23169,69080910,MS (Elec biased) Maintenance Engineer / Superv...,Harrogate,non_specified,permanent,Jenrick Engineering,Engineering Jobs,17004,cv-library.co.uk,20132108T120000,20131119T120000


It seems though these OpenDates are invalid, if the month and day get reversed they are all valid date.

Thus we decide to reverse them.

In [ ]:
df1[df1.OpenDate.isin(invalid_series)]['OpenDate'].apply(lambda x: re.sub(,,x))# need close look at re.sub

In [40]:
df1 = df1[~df1.OpenDate.isin( invalid_series)]

In [42]:
#transform the string representation of date to type datetime for more flexibility
df1[['OpenDate','CloseDate']] = df1[['OpenDate','CloseDate']].apply(lambda x: pd.to_datetime(x))

In [46]:
df1[~df1.Company.notna()]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
51,34479775,Credit Controller,The City,non_specified,permanent,NaN,Accounting & Finance Jobs,24000,hays.co.uk,2012-03-26 00:00:00,2012-04-25 00:00:00
55,34790490,Java Developer,UK,non_specified,permanent,NaN,IT Jobs,33996,theitjobboard.co.uk,2012-02-03 12:00:00,2012-03-04 12:00:00
66,38209887,Transaction Services Executive,Bristol,non_specified,permanent,NaN,Accounting & Finance Jobs,37500,hays.co.uk,2012-03-15 15:00:00,2012-05-14 15:00:00
67,38209888,Audit Executive,Bristol,non_specified,permanent,NaN,Accounting & Finance Jobs,33996,hays.co.uk,2012-06-10 12:00:00,2012-09-08 12:00:00
68,38209893,Audit Assistant Manager,Bristol,non_specified,permanent,NaN,Accounting & Finance Jobs,35004,hays.co.uk,2012-09-13 12:00:00,2012-11-12 12:00:00
70,38758190,Account Executive,Basingstoke,non_specified,permanent,NaN,IT Jobs,18000,hays.co.uk,2012-03-14 12:00:00,2012-06-12 12:00:00
78,40880208,Compliance Manager,Salisbury,non_specified,permanent,NaN,Accounting & Finance Jobs,37500,hays.co.uk,2012-11-19 12:00:00,2013-01-18 12:00:00
81,41879934,Future Leaders Development Programme Technology,UK,non_specified,permanent,NaN,IT Jobs,36000,theitjobboard.co.uk,2012-08-15 15:00:00,2012-10-14 15:00:00
105,46626508,Support Worker Job Chepstow,UK,full_time,non_specified,NaN,Healthcare & Nursing Jobs,12348,careworx.co.uk,2013-04-07 00:00:00,2013-05-07 00:00:00
106,46626509,Support Worker Job Abergavenny,UK,full_time,non_specified,NaN,Healthcare & Nursing Jobs,14256,careworx.co.uk,2013-06-14 00:00:00,2013-09-12 00:00:00


## 3. Summary
Give a short summary of your work done above, such as your findings.